In [35]:
import random
import itertools
from operator import attrgetter

In [2]:
class Customer:
    def __init__(self,arrival_time,service_start_time,service_time):
        self.arrival_time=arrival_time
        self.service_start_time=service_start_time
        self.service_time=service_time
        self.service_end_time = self.service_start_time + self.service_time
        self.wait = self.service_start_time - self.arrival_time

In [94]:
class Server:
    def __init__(self,free,service_start_time):
        self.free=free
        self.service_start_time=service_start_time
        self.service_end_time = 0

In [4]:
#a simple function to sample from negative exponential
def neg_exp(lambd):
    return random.expovariate(lambd)

In [82]:
def QSim_1(arv, p, simulation_time, explicit = False):
    """
    This is the main function to call to simulate an MM1 queue.
    """

    #Initialise run
    t=0

    #Initialise empty list to hold all data
    Customers=[]
    
    #The actual simulation happens here:
    while t < simulation_time/60:

        #calculate arrival date and service time for new customer
        if len(Customers)==0:
            arrival_time = neg_exp(arv)
            service_start_time = arrival_time
        else:
            arrival_time += neg_exp(arv)
            service_start_time = max(arrival_time, Customers[-1].service_end_time)
        
        service_time = neg_exp(60/p)

        #create new customer
        Customers.append(Customer(arrival_time,service_start_time,service_time))

        #increment clock till next end of service
        t=arrival_time
    
    total_waits=[a.wait for a in Customers]
    Tq = sum(total_waits)/len(total_waits)
    
    if explicit == False:
        print('Average waiting time: %d minutes' % (Tq*60))
    else:
        for i in range(0,len(Customers)-1):
            print('Customer %d: Arrival %d, Service start: %d, Service time: %d minutes' % (i+1, Customers[i].arrival_time*60,
                                                             Customers[i].service_start_time*60, Customers[i].service_time*60))

In [28]:
QSim_1(13, 25, 60)

Average waiting time: 65 minutes


In [83]:
QSim_1(13, 25, 60, True)

Customer 1: Arrival 4, Service start: 4, Service time: 8 minutes
Customer 2: Arrival 6, Service start: 12, Service time: 15 minutes
Customer 3: Arrival 7, Service start: 27, Service time: 9 minutes
Customer 4: Arrival 7, Service start: 36, Service time: 51 minutes
Customer 5: Arrival 15, Service start: 88, Service time: 6 minutes
Customer 6: Arrival 19, Service start: 95, Service time: 62 minutes
Customer 7: Arrival 24, Service start: 157, Service time: 11 minutes
Customer 8: Arrival 27, Service start: 169, Service time: 4 minutes
Customer 9: Arrival 34, Service start: 173, Service time: 5 minutes
Customer 10: Arrival 35, Service start: 178, Service time: 18 minutes
Customer 11: Arrival 42, Service start: 197, Service time: 20 minutes
Customer 12: Arrival 51, Service start: 217, Service time: 29 minutes
Customer 13: Arrival 54, Service start: 246, Service time: 11 minutes
Customer 14: Arrival 59, Service start: 258, Service time: 4 minutes


In [ ]:
QSim_1(13, 25, 10000, True)

For multiple servers

In [127]:
def QSim_m(arv = False, p = False, servers = False, simulation_time=False, explicit = "Tq"):
    """
    This is the main function to call to simulate an MM1 queue.
    """

    #If parameters are not input prompt
    if not arv:
        arv =input('Arrival per hour (1/a): ')
    if not p:
        p = input('Service time in minutes: ')
    if not servers:
        servers = input('Number of servers: ')
    if not simulation_time:
        simulation_time = input('Total simulation time: ')

    #Initialise run
    t=0

    #Initialise empty list to hold all data
    Customers=[]
    
    #Setting the servers
    Servers = []
    for i in range(servers): Servers.append(Server(1,0))
    
    #The actual simulation happens here:
    while t < int(simulation_time)/60:
        
        #calculate arrival date and service time for new customer
        if len(Customers)==0:
            arrival_time = neg_exp(int(arv))
            service_start_time = arrival_time
        else:
            arrival_time += neg_exp(int(arv))
              
        service_time = neg_exp(60/int(p))
        
        #Check and compare server service_end_time to t:
        for ser in Servers:
            if ser.service_end_time < t:
                ser.free = 1
                
        #Check the servers to assign:
        #Find the server that finishes the earliest
        m = min(x.service_end_time for x in Servers)
        for ser in Servers:
            if ser.service_end_time == m:
                service_start_time = max(arrival_time, m)
                ser.service_start_time = max(arrival_time, m)
                ser.service_end_time = ser.service_start_time + service_time
                ser.free = 0
                break      

        #create new customer
        Customers.append(Customer(arrival_time,service_start_time,service_time))
        
        #increment clock till next end of service
        t=arrival_time
    
    total_waits=[a.wait for a in Customers]
    Tq = sum(total_waits)/len(total_waits)
    
    if explicit == 'Tq':
        print('Average waiting time: %d minutes' % (Tq*60))
    elif explicit == 'Flow':
        for i in range(0,len(Customers)-1):
            print('Customer %d: Arrival %d, Service start: %d, Service time: %d minutes, Service end time: %d' % (i+1, 
                                                    Customers[i].arrival_time*60, Customers[i].service_start_time*60, 
                                                    Customers[i].service_time*60, Customers[i].service_end_time*60))
    elif explicit == 'Spill':
        spill = sum([1 if x.service_start_time > t else 0 for x in Customers])
        print('Customers not served: %d' % spill)
    elif explicit == 'Spsi':
        spill = sum([1 if x.service_start_time > t else 0 for x in Customers])
        return spill

In [141]:
def QSim_spill(arv = False, p = False, servers = False, simulation_time=False, n = 1000):
    simu = []
    i = 0
    while i < n:
        simu.append(float(QSim_m(arv, p, servers, simulation_time, explicit = "Spsi")))
        i += 1
    print('Average customer spillover: %d' % (sum(simu)/len(simu)))
    print('Max customer spillover: %d' % (max(simu)))
    print('Min customer spillover: %d' % (min(simu)))

In [135]:
QSim_m(13, 25, 3, 60, 'Spill')

Customers not served: 7


In [136]:
QSim_m(13, 25, 3, 60, 'Spsi')

5

In [149]:
QSim_spill(arv = 13, p = 25, servers = 3, simulation_time = 60, n= 10000)

Average customer spillover: 5
Max customer spillover: 20
Min customer spillover: 0
